In [1]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency
import seaborn
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/hr_raw_data_definitivo.csv', index_col=0)
df.head()

,employee_number,attrition,job_level,job_role,department,monthly_income,salary,percent_salary_hike,training_times_last_year,stock_option_level,...,age,gender,marital_status,education,education_field,num_companies_worked,total_working_years,years_at_company,years_since_last_promotion,years_with_curr_manager
0,1,No,5,research director,research & development,16280.83,195370.00,13,5,0,...,52,M,NaN,3,other,7,NaN,20,15,15
1,2,No,5,manager,general,16665.83,199990.00,14,5,1,...,53,M,NaN,4,life sciences,0,34.0,33,11,9
2,3,No,5,manager,research & development,16026.67,192320.00,11,3,0,...,43,M,married,2,technical degree,1,22.0,22,11,15
3,4,No,4,research director,research & development,14307.50,171690.00,19,2,2,...,48,F,married,4,medical,3,NaN,20,5,6
4,5,No,4,sales executive,sales,12783.92,153407.04,12,5,1,...,47,F,divorced,3,technical degree,2,NaN,19,2,8


In [4]:
grupo_control = df[df['satisfaction'] == 'satisfied']

In [5]:
grupo_variante = df[df['satisfaction'] == 'not satisfied']

In [15]:
grupo_control_porcentaje_rotacion = grupo_control[grupo_control['attrition'] == 'Yes']["employee_number"].count()/grupo_control.shape[0]*100
print(grupo_control_porcentaje_rotacion)

13.668341708542714


In [16]:
grupo_variante_porcentaje_rotacion = grupo_variante[grupo_variante['attrition'] == 'Yes']["employee_number"].count()/grupo_variante.shape[0]*100
print(grupo_variante_porcentaje_rotacion)

19.87075928917609
